In [ ]:
sc.install_pypi_package("pybind11==2.10.3")
sc.install_pypi_package("numpy==1.19.0")
sc.install_pypi_package("Pillow==8.2")
sc.install_pypi_package("Cython==0.29.33")
sc.install_pypi_package("scipy==1.2.0")
sc.install_pypi_package("pythran==0.12.1")
sc.install_pypi_package("pandas==1.0.0")
sc.install_pypi_package("matplotlib==3.3.0")
sc.install_pypi_package("lifelines==0.27.4")
sc.install_pypi_package("s3fs==0.4.2")
sc.install_pypi_package("boto3==1.26.59")
sc.install_pypi_package("joblib==1.2.0")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from itertools import groupby
from operator import itemgetter

from lifelines import CoxPHFitter

import pandas as pd
import boto3
import os
import pickle
import tempfile
import joblib
import io
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

MongoUrl = "mongodb+srv://xxxxxxxxxxxxxxxxxxxxxx/"

In [ ]:
spark = SparkSession \
    .builder \
    .appName("FinancialSheets_ML_Training") \
    .config("spark.cores.max", 6) \
    .config("spark.executor.cores", 6) \
    .config("spark.executor.memory", "36g") \
    .getOrCreate()

In [ ]:
df = spark.read.format("mongodb") \
    .option("spark.mongodb.read.connection.uri", MongoUrl) \
    .option("spark.mongodb.write.connection.uri", MongoUrl) \
    .option("database", "coreEngine") \
    .option("collection", "ReportFeaturesTest") \
    .load()

In [ ]:
features = df.toPandas()

In [ ]:
hf_features = features.drop(["_id"], axis=1)
hf_features["event"] = hf_features["event"].fillna(0).astype(bool)
hf_features = hf_features.sort_values(["stockCode", "rceptNo"])
hf_features = hf_features[hf_features["period"] >= 1.0]

In [ ]:
columns = [
            "DSR01", "DSR02", "DSR03", "DSR04", "DSR05",
            "CLTL", "EBTIN2", "INTL2", "LNSL", "LNTA", "MB2", "NIGR2", "NISL", "RETA", 
            "SLEQ", "SLFA", "TLEQ"]
#             "FFOEQ", "FFOTL", "CACL", "EQTA", "INSL", "CATA", "TLTA", "INTL", 
#             "MB", "NIGR", "FAGR", "EBTIN", "CLCA", "NEGBE", "CLGR", 
         

    
for column in columns:
    hf_features[column]=hf_features.groupby(["bsnsYear", "reprtCode"])[column].apply(lambda x:x.fillna(0))

In [ ]:
hf_features = hf_features.dropna()

In [ ]:
boundry = "2018"
boudnry2 = "2024"
indexColumn = ["stockCode", "corpCls", "corpCode", "reprtCode", "rceptNo", "stockName", "updateDate", "bsnsYear", "quarter"]

df_train_set = hf_features[(hf_features["bsnsYear"] >= boundry) & (hf_features["bsnsYear"] < boudnry2)].drop_duplicates()
df_train_set = df_train_set.set_index(indexColumn)

In [ ]:
model = CoxPHFitter(penalizer=0.0001, l1_ratio=1)  #pernalizer가 0인 것은 정규화 없는 모델, 0보다 클수록 정규화 효과 강화. 모델을 더 일반화 시킨다는 것.
model.fit(df_train_set, duration_col="period", event_col="event", fit_options=dict(step_size=0.2))

# period 는 estDt부터 재무제표 등록 일자까지의 기간

'''
** event 변수는 '법인세차감전이익'(t) + '법인세차감전이익'(t-1) < 0 을 떄 1,

val disclosure  #몽고db에서 불러와서 disclosure라는 데이터프레임 생성.
    spark.read.format("mongodb")
    .withColumn("event", lit(1)) #event 컬럼 생성하고 1로 초기화

val fs_features    #fs_preprocess 데이터프레임으로 새로운 데이터프레임을 만드는 것.
    fs_preprocess   
    .withColumn("event", when(col("ProfitLossBeforeTax") + lag(col("ProfitLossBeforeTax"), offset=1).over(partition)<0, 1).otherwise(col("event")))

---------------------------------------
cox 비례 위험 모형
event : 생존/사망 또는 잔류/사퇴와 같은 관심 경험 // 우리는 최근 2분기의 영업이익의 합계가 <0 이거나 불성실공시인 경우.
생존시간 : event가 발생할때까지의 기간 // 우리는 설립일부터 updateDate까지의 기간

hazard : t-1에 살았던 사람이 t시점에 죽을 확률
H(t) = hazard 누적  => 일정하지 않고. 시간에 따라 변함
따라서 
1. log rank test (x가 하나일 때 주로 사용)
    HR(t) = H(위험인자 노출군)/H(비노출군)  => 평균은 HRMC라고 하고 이게 1이면, 시험군과 대조근의 차이가 없는 것.  

2. cox's propotional hazard medel
    HR(t)는 시간에 대해 일정하다고 가정.
    이 가정에 맞지 않는 데이터라면 결과 왜곡 가능성 높음.

    ##정확한 이해를 위해 통계학 수업 또는 유튜브 설명 듣기!!
    

'''

In [ ]:
model.print_summary()

In [ ]:
s3 = boto3.client(
    "s3",
    region_name="ap-northeast-2",
    aws_access_key_id="xxxxxxxxxxxxxxxxxxxxxx",
    aws_secret_access_key="xxxxxxxxxxxxxxxxxxxxxx")

In [ ]:
"""
f = io.BytesIO()
joblib.dump(model, f)
f.seek(0)
s3.put_object(Bucket="penta-engine", Key="FinancialSheetsModelNew.pkl", Body=f)
"""

In [ ]:
spark.stop()